# Example on how to import URL resources and merge to 2 or more DataFrames.

In [1]:
import pandas as pd
import numpy as np

In [5]:
#To import resources from a web site, use 'url'.
url = 'https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/android_devices.csv'

#Github csv is read as url.

android_devices = pd.read_csv(url)
android_devices.head(2)

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21


In [6]:
url = 'https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/user_device.csv'

user_devices = pd.read_csv(url)
user_devices.head(2)

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3


In [7]:
url = 'https://raw.githubusercontent.com/shanealynn/Pandas-Merge-Tutorial/master/user_usage.csv'

user_usage = pd.read_csv(url)
user_usage.head(2)

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788


# We would like to determine if the usage patterns for users differ between different devices.

In [8]:
#Form a single dataframe with columns for "user usage" (calls/month, text/month, etc)
#& columns from "user devices" (model, manufacturer, etc). 
#Need to “merge” (or 'join') our datasets into 1 dataset for analysis.

In [10]:
result = pd.merge(user_usage, user_devices[['use_id', 'platform', 'device']], on='use_id')
result.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [16]:
#Joining the 'device details' to the result of the 'User usage' & 'user device' 

#First add the platform & device columns to the user usage.
result = pd.merge(user_usage, user_devices[['use_id', 'platform', 'device']], on='use_id', how='left')
result.head()

#Now based on the "device column" in the result, match the "Model" column in devices.
android_devices.rename(columns = {"Retail Branding": "manufacturer"}, inplace=True)
result = pd.merge(result, android_devices[['manufacturer', 'Model']],
                 left_on='device',
                 right_on='Model',
                 how='left')
result.head(2)


,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device,manufacturer,Model
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung,SM-G930F


In [23]:
#look at basic statistics for mean usage for users based on device manufacturer.
result.groupby("manufacturer").agg({ "use_id":"count",
                                "outgoing_mins_per_month": "mean",
                                "outgoing_sms_per_month": "mean",
                                "monthly_mb": "mean"})

,use_id,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb
manufacturer,,,,
HTC,44,299.842955,93.059318,5144.077955
Huawei,3,81.526667,9.500000,1561.226667
LGE,2,111.530000,12.760000,1557.330000
Lava,2,60.650000,261.900000,12458.670000
Lenovo,2,215.920000,12.930000,1557.330000
Motorola,16,95.127500,65.666250,3946.500000
OnePlus,6,354.855000,48.330000,6575.410000
Samsung,108,191.010093,92.390463,4017.318889
Sony,16,177.315625,40.176250,3212.000625
